# 한영 번역 서비스 만들기 M2M100

In [1]:
# !pip install sentencepiece

In [3]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

ko_text = "이것은 한국어/영어 번역기입니다. m2m100 모델로 만들었어요. 참 좋네요"
eng_text = "This is a Korean-English translator. It's an m2m100 model. That's great"

model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")


tokenizer.src_lang = "ko"
encoded_ko = tokenizer(ko_text, return_tensors="pt")
generated_tokens = model.generate(**encoded_ko, forced_bos_token_id=tokenizer.get_lang_id("en"))
translated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
print(translated_text)


tokenizer.src_lang = "en"
encoded_en = tokenizer(eng_text, return_tensors="pt")
generated_tokens = model.generate(**encoded_en, forced_bos_token_id=tokenizer.get_lang_id("ko"))
translated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
print(translated_text)

This is a Korean/English translator. made in m2m100 model. great
이것은 한국어 - 영어 번역기입니다.이것은 m2m100 모델입니다.


GPT가 짜준 코드

In [6]:
import gradio as gr
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

# Load the model and tokenizer
model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

def translate_text(input_text, source_lang):
    if source_lang == "Korean (ko)":
        tokenizer.src_lang = "ko"
        encoded = tokenizer(input_text, return_tensors="pt")
        generated_tokens = model.generate(**encoded, forced_bos_token_id=tokenizer.get_lang_id("en"))
        translated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    else:
        tokenizer.src_lang = "en"
        encoded = tokenizer(input_text, return_tensors="pt")
        generated_tokens = model.generate(**encoded, forced_bos_token_id=tokenizer.get_lang_id("ko"))
        translated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    
    return translated_text

with gr.Blocks() as app:
    gr.Markdown("# Korean-English Translation Service")
    
    with gr.Row():
        source_lang = gr.Radio(["Korean (ko)", "English (en)"], label="Select Source Language")
        input_text = gr.Textbox(label="Input Text", placeholder="Enter text to translate...")
        output_text = gr.Textbox(label="Translated Text", interactive=False)
        translate_btn = gr.Button("Translate")
        
        translate_btn.click(
            fn=translate_text,
            inputs=[input_text, source_lang],
            outputs=output_text
        )
    
    gr.Markdown("### Example Inputs:")
    gr.Examples(
        examples=[
            ["이것은 번역기입니다.", "Korean (ko)"],
            ["This is a translator.", "English (en)"]
        ],
        inputs=[input_text, source_lang],
        outputs=output_text,
        fn=translate_text
    )

app.launch()

/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


IMPORTANT: You are using gradio version 3.40.0, however version 4.29.0 is available, please upgrade.
--------


Directml 활용하면 더 빠르게 결과가 나옴

In [2]:
import torch_directml  # Import torch-directml for DirectML device
import torch  # Import PyTorch
import gradio as gr
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

# Initialize the DirectML device
dml_device = torch_directml.device()

# Initialize the model and tokenizer
model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M").to(dml_device)
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

def translate_text(input_text, source_lang):
    if source_lang == "Korean (ko)":
        tokenizer.src_lang = "ko"
        encoded = tokenizer(input_text, return_tensors="pt")
        generated_tokens = model.generate(**encoded, forced_bos_token_id=tokenizer.get_lang_id("en"))
        translated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    else:
        tokenizer.src_lang = "en"
        encoded = tokenizer(input_text, return_tensors="pt")
        generated_tokens = model.generate(**encoded, forced_bos_token_id=tokenizer.get_lang_id("ko"))
        translated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    
    return translated_text

with gr.Blocks() as app:
    gr.Markdown("# Korean-English Translation Service")
    
    with gr.Row():
        source_lang = gr.Radio(["Korean (ko)", "English (en)"], label="Select Source Language")
        input_text = gr.Textbox(label="Input Text", placeholder="Enter text to translate...")
        output_text = gr.Textbox(label="Translated Text", interactive=False)
        translate_btn = gr.Button("Translate")
        
        translate_btn.click(
            fn=translate_text,
            inputs=[input_text, source_lang],
            outputs=output_text
        )
    
    gr.Markdown("### Example Inputs:")
    gr.Examples(
        examples=[
            ["이것은 번역기입니다.", "Korean (ko)"],
            ["This is a translator.", "English (en)"]
        ],
        inputs=[input_text, source_lang],
        outputs=output_text,
        fn=translate_text
    )

app.launch(inline = False, share = True)

/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/gradio_client/documentation.py:106: UserWarning: Could not get documentation group for <class 'gradio.mix.Parallel'>: No known documentation group for module 'gradio.mix'
  warnings.warn(f"Could not get documentation group for {cls}: {exc}")
/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/gradio_client/documentation.py:106: UserWarning: Could not get documentation group for <class 'gradio.mix.Series'>: No known documentation group for module 'gradio.mix'
  warnings.warn(f"Could not get documentation group for {cls}: {exc}")
/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `cle

Running on local URL:  http://127.0.0.1:7860
IMPORTANT: You are using gradio version 3.40.0, however version 4.29.0 is available, please upgrade.
--------
Running on public URL: https://7b185a4fdafd69f896.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/gradio/routes.py", line 488, in run_predict
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/gradio/blocks.py", line 1427, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/gradio/blocks.py", line 1109, in call_function
    prediction = await anyio.to_thread.run_sync(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/anyio/_backends/_asyncio.py", line 

In [3]:
app.close()

Closing server running on port: 7860


## tts까지 붙여보기
- gtts로 하려고 했는데 directml 사용하니까 컴퓨터가 계속 죽어버려서 하지 않기로 함

In [4]:
import gradio as gr
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
import torch_directml  # Import torch-directml for DirectML device
import torch  # Import PyTorch
from gtts import gTTS  # Import Google Text-to-Speech
import os

# Initialize the DirectML device
dml_device = torch_directml.device()

# Initialize the model and tokenizer
model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M").to(dml_device)
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

# List of available languages with their language codes
languages = {
    "Korean": "ko",
    "English": "en",
    "French": "fr",
    "Spanish": "es",
    "German": "de",
    "Japanese": "ja",
    "Chinese (Simplified)": "zh",
    "Russian": "ru",
    # Add more languages as needed
}

# Translation function
def translate(text, source_lang, target_lang):
    tokenizer.src_lang = languages[source_lang]
    encoded_text = tokenizer(text, return_tensors="pt").to(dml_device)  # Send input tensors to DirectML device
    generated_tokens = model.generate(**encoded_text, forced_bos_token_id=tokenizer.get_lang_id(languages[target_lang]))
    translated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    return translated_text

# TTS function using gTTS
def text_to_speech(text, target_lang):
    tts = gTTS(text, lang=languages[target_lang])
    audio_file = "output.mp3"
    tts.save(audio_file)
    return audio_file

# Gradio app
with gr.Blocks() as app:
    gr.Markdown("# Multilingual Translation and TTS Service")

    with gr.Row():
        source_lang = gr.Dropdown(label="Source Language", choices=list(languages.keys()), value="Korean")
        target_lang = gr.Dropdown(label="Target Language", choices=list(languages.keys()), value="English")

    input_text = gr.Textbox(label="Input Text")
    output_text = gr.Textbox(label="Translated Text")

    # Add buttons for translation and TTS
    with gr.Row():
        translate_btn = gr.Button("Translate")
        tts_btn = gr.Button("🔊", elem_id="tts_button")  # Button with a speaker icon for TTS

    # Define the output for TTS (audio file)
    output_audio = gr.Audio(label="TTS Output", type="filepath")

    # Button click events
    translate_btn.click(fn=translate, inputs=[input_text, source_lang, target_lang], outputs=output_text)
    tts_btn.click(fn=text_to_speech, inputs=[output_text, target_lang], outputs=output_audio)

app.launch(inline=False, share=True)


ModuleNotFoundError: No module named 'gtts'

In [3]:
app.close()

Closing server running on port: 7860


## tts 붙여보기
- 허깅페이스에 있는 거 가져와서 해보기

In [6]:
# !pip install --upgrade pip
# !pip install --upgrade transformers sentencepiece datasets[audio]

In [3]:
# 허깅페이스에 있는 코드 예시
# from transformers import pipeline
# from datasets import load_dataset
# import soundfile as sf
# import torch

# synthesiser = pipeline("text-to-speech", "microsoft/speecht5_tts")

# embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
# speaker_embedding = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)
# # You can replace this embedding with your own as well.

# speech = synthesiser("Hello, my dog is cooler than you!", forward_params={"speaker_embeddings": speaker_embedding})

# sf.write("speech.wav", speech["audio"], samplerate=speech["sampling_rate"])

In [4]:
# 허깅페이스에 있는 코드 예시
# from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
# from datasets import load_dataset
# import torch
# import soundfile as sf

# processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
# model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
# vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

# inputs = processor(text="Hello, my dog is cute.", return_tensors="pt")

# # load xvector containing speaker's voice characteristics from a dataset
# embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
# speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

# speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)

# sf.write("speech.wav", speech.numpy(), samplerate=16000)

In [5]:
# tts까지 같이 있는 버전
import gradio as gr
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer, SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
import torch
import soundfile as sf
from datasets import load_dataset

# 번역 모델 설정
model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M").to("cpu")
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

# TTS 모델 설정
processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
tts_model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

# 스피커 임베딩 데이터셋 로드
embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
speaker_embedding = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

# 지원하는 언어 목록
languages = {
    "Korean": "ko",
    "English": "en",
    "French": "fr",
    "Spanish": "es",
    "German": "de",
    "Japanese": "ja",
    "Chinese (Simplified)": "zh",
    "Russian": "ru",
}

# 번역 함수
def translate(text, source_lang, target_lang):
    tokenizer.src_lang = languages[source_lang]
    encoded_text = tokenizer(text, return_tensors="pt").to("cpu")
    generated_tokens = model.generate(**encoded_text, forced_bos_token_id=tokenizer.get_lang_id(languages[target_lang]))
    translated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    return translated_text

# TTS 함수
def text_to_speech(text):
    inputs = processor(text=text, return_tensors="pt")
    speech = tts_model.generate_speech(inputs["input_ids"], speaker_embedding, vocoder=vocoder)
    sf.write("output.wav", speech.numpy(), samplerate=16000)
    return "output.wav"

# Gradio 인터페이스 설정
with gr.Blocks() as app:
    gr.Markdown("# Translation and TTS Service")

    with gr.Row():
        source_lang = gr.Dropdown(label="Source Language", choices=list(languages.keys()), value="Korean")
        target_lang = gr.Dropdown(label="Target Language", choices=list(languages.keys()), value="English")

    input_text = gr.Textbox(label="Input Text")
    output_text = gr.Textbox(label="Translated Text")

    # 번역 및 TTS 버튼 추가
    with gr.Row():
        translate_btn = gr.Button("Translate")
        tts_btn = gr.Button("🔊", elem_id="tts_button")

    output_audio = gr.Audio(label="TTS Output", type="filepath")

    # 버튼 클릭 이벤트 설정
    translate_btn.click(fn=translate, inputs=[input_text, source_lang, target_lang], outputs=output_text)
    tts_btn.click(fn=text_to_speech, inputs=output_text, outputs=output_audio)

app.launch(inline=False, share=True)

/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/gradio_client/documentation.py:106: UserWarning: Could not get documentation group for <class 'gradio.mix.Parallel'>: No known documentation group for module 'gradio.mix'
  warnings.warn(f"Could not get documentation group for {cls}: {exc}")
/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/gradio_client/documentation.py:106: UserWarning: Could not get documentation group for <class 'gradio.mix.Series'>: No known documentation group for module 'gradio.mix'
  warnings.warn(f"Could not get documentation group for {cls}: {exc}")
/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884

Running on local URL:  http://127.0.0.1:7860
IMPORTANT: You are using gradio version 3.40.0, however version 4.29.0 is available, please upgrade.
--------
Running on public URL: https://ea2301db0d8f4d055c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [7]:
app.close()

Closing server running on port: 7860
